In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%%capture
! pip install transformers

In [ ]:
import torch
from transformers import AutoModel, AutoTokenizer
import pandas as pd
import numpy as np

In [ ]:
def score(a, b):
    if len(a.shape) == 1: a = a.unsqueeze(0)
    if len(b.shape) == 1: b = b.unsqueeze(0)

    a_norm = a / a.norm(dim=1)[:, None]
    b_norm = b / b.norm(dim=1)[:, None]
    return torch.mm(a_norm, b_norm.transpose(0, 1)) * 100

In [ ]:
model = AutoModel.from_pretrained('BM-K/KoSimCSE-bert-multitask')  
tokenizer = AutoTokenizer.from_pretrained('BM-K/KoSimCSE-bert-multitask')  

In [ ]:
data = pd.read_csv('./Books.csv', encoding = 'utf-8')[:20]

In [ ]:
dic = {'title':[], 'similarity':[]}
sentences = []

for i in range(len(data)) :
    sentences.append(data['overview'][i])

for i in range(len(data)) :
    if i % 10 == 0 :
        print(i, '/', len(data))
    try : 
        tmp = []
        tmp_sentence = [sentences[i]]

        for j in range(len(sentences)) :
            if i != j :
                tmp_sentence.append(sentences[j])

                inputs = tokenizer(tmp_sentence, padding = True, truncation = True, return_tensors = 'pt')
                embeddings, _ = model(**inputs, return_dict = False)

                score = score(embeddings[0][0], embeddings[1][0])[0][0].item()
                tmp.append(score)

        dic['title'].append(data['title'][i])
        dic['similarity'].append(np.mean(tmp))
    
    except :
        print(i, j)

pd.DataFrame(dic)